# Impala Annotation Pipeline

This pipeline begins with the existing table of distcinct variants found on impala and will be modified for adding new variants. 

## Connect to Required Impala Tables

In [49]:
from impala.dbapi import connect
from impala.util import as_pandas
import pandas as pd
import sys
sys.path.append('/Users/selasady/impala_scripts/parse_impala.py')
import time
import csv
import os
import subprocess as subp

## Pull down distinct variant table from imapala

In [4]:
# query to left join nbs_annotated variants with DANN scores
distinct_query = """
    SELECT DISTINCT * FROM users_selasady.distinct_test
    """
# run query on impala
conn=connect(host='glados19', port=21050, timeout=120)
cur = conn.cursor()
cur.execute(distinct_query)

# store results as pandas data frame
distinct_df = as_pandas(cur)
cur.close()
conn.close()

## Save impala table as VCF

In [55]:
# view table structure
print distinct_df.head()

  chrom        pos ref alt           var_id
0    15   25207797   C   T  15:25207797:C:T
1    16    7162330   A   T   16:7162330:A:T
2     4  146853510   C   T  4:146853510:C:T
3    13   58314331   G   T  13:58314331:G:T
4     1  161484124   A   C  1:161484124:A:C


In [52]:
# disable extraneous pandas warning
pd.options.mode.chained_assignment = None

# create vcf header  
pimp.create_header('test.vcf')

# crate vcf file
pimp.impala_to_vcf(distinct_df, 'test.vcf')

## Run VCF Through SnpEff

In [53]:
# Run VCF through SnpEff
def vcf_to_snpeff(input_vcf, snpeff_path, vcf_name, out_name):
    if os.path.exists(snpeff_path):
        # command to run snpeff
        snpeff_cmd = 'java -Xmx4g -jar {}  -t -v -noStats GRCh37.74 {} > {}_snpeff.vcf'.format(
            snpeff_path, vcf_name, out_name)
        # run snpeff on vcf file
        snpeff_process = subp.Popen(snpeff_cmd, shell=True)
        snpeff_process.wait()
    else:
        print "Make sure you entered the correct path to snpEff.jar"

# path to snpeff jar
snpeff_path = '/Users/selasady/tools/snpEff/snpEff.jar'   
vcf_to_snpeff(distinct_df, snpeff_path, './test.vcf', 'annotated.vcf')

## Convert VCF to BED/BIM/FAM